In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns
import os

# Mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)

In [3]:
# Rutas
RAW_PATH = "../data/original/US_Accidents_March23.csv"
PROCESSED_PATH = "../data/processed/accidents_clean.csv"

In [4]:
# 1. Cargar el dataset
df = pd.read_csv(RAW_PATH)
print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

# 2. Vista previa
display(df.head())

Dataset cargado: 7728394 filas, 46 columnas


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [5]:
# 3. Tipos de datos
print("\nTipos de datos:")
print(df.dtypes.value_counts())


Tipos de datos:
object     20
bool       13
float64    12
int64       1
Name: count, dtype: int64


In [6]:
# 4. Verificar la estructura y el tipo de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [7]:
# 5. Nulos
nulls = df.isnull().mean().sort_values(ascending=False) * 100
nulls = nulls.round(2)

print("\nPorcentaje de valores nulos por columna:\n")
for col, pct in nulls.items():
    print(f"{col:<25} {pct:>6.2f} %")



Porcentaje de valores nulos por columna:

End_Lng                    44.03 %
End_Lat                    44.03 %
Precipitation(in)          28.51 %
Wind_Chill(F)              25.87 %
Wind_Speed(mph)             7.39 %
Visibility(mi)              2.29 %
Wind_Direction              2.27 %
Humidity(%)                 2.25 %
Weather_Condition           2.24 %
Temperature(F)              2.12 %
Pressure(in)                1.82 %
Weather_Timestamp           1.56 %
Sunrise_Sunset              0.30 %
Civil_Twilight              0.30 %
Astronomical_Twilight       0.30 %
Nautical_Twilight           0.30 %
Airport_Code                0.29 %
Street                      0.14 %
Timezone                    0.10 %
Zipcode                     0.02 %
City                        0.00 %
Description                 0.00 %
ID                          0.00 %
Distance(mi)                0.00 %
Start_Lng                   0.00 %
Source                      0.00 %
Severity                    0.00 %
Start_Time  

In [8]:
# 6. Eliminar columnas sin utilidad o con >30% de nulos

# Columnas irrelevantes
irrelevant_cols = [
    'ID',                  # Identificador único
    'Source',              # Fuente del dato (no aporta al análisis)
    'Description',         # Descripción libre, texto largo
    'Street',              # Calle completa, texto no procesado
    'Zipcode',             # Código postal, redundante con city/county/state
    'Country',             # Siempre es US
    'Weather_Timestamp',   # Timestamp diferente del de Start_Time
    'Start_Lat',           # Latitud   
    'Start_Lng',           # Longitud
    'End_Lng',             # Latitud   
    'End_Lng',              # Longitud
    'End_Time',
    'Astronomical_Twilight',
    'Nautical_Twilight',
    'Airport_Code'
]

# Eliminar del dataframe
df = df.drop(columns=irrelevant_cols)

# 7. Eliminar filas de columnas con pocos nulos <5%
columns_low_nulls = [
    'Visibility(mi)', 'Wind_Direction', 'Humidity(%)',
    'Weather_Condition', 'Temperature(F)', 'Pressure(in)',
    'Sunrise_Sunset', 'Civil_Twilight', 'Timezone'
]
# Eliminar filas con nulos en esas columnas (seguro, dataset grande)
df = df.dropna(subset=columns_low_nulls)

# Columnas con mas nulos → imputar
columns_to_impute = ['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)']

# Imputar con la media (porque son valores numéricos)
imputer = SimpleImputer(strategy='mean')
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

Para gestionar los registros con campos nulos, decidí según la importancia del campo y el número de nulos existentes:

    - Los campos que tienen pocos registrso nulos, eliminaré esos registros ya que debido al tamaño del dataset (mas de 7 millones de registros) no deberían influir en el análisis.
    
    - Los campos con muchos registros nulos se les imputara un valor basado en la media o mediana o modo (valor mas frecuente en variables categoricad) para que no afecten el resultado del análisis

In [ ]:
# 8. Convertir columnas de fecha a datetime
for col in ["Start_Time"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")


In [12]:
# Ver el número de valores únicos por columna
unique_counts = df.nunique()

# Ver los valores únicos de cada columna 
unique_values = {col: df[col].unique() for col in df.columns}

print("Número de valores únicos por columna:")
print(unique_counts)

print("\nValores únicos por columna:")
for col, values in unique_values.items():
    print(f"\n{col}:")
    print(values[:10])  # Muestra solo los primeros 10 valores únicos para evitar una salida muy larga

Número de valores únicos por columna:
Severity                   4
Start_Time           5365040
End_Lat              1509340
Distance(mi)           21837
City                   12237
County                  1813
State                     49
Timezone                   4
Temperature(F)           845
Wind_Chill(F)            994
Humidity(%)              100
Pressure(in)            1133
Visibility(mi)            87
Wind_Direction            24
Wind_Speed(mph)          180
Precipitation(in)        289
Weather_Condition        142
Amenity                    2
Bump                       2
Crossing                   2
Give_Way                   2
Junction                   2
No_Exit                    2
Railway                    2
Roundabout                 2
Station                    2
Stop                       2
Traffic_Calming            2
Traffic_Signal             2
Turning_Loop               1
Sunrise_Sunset             2
Civil_Twilight             2
dtype: int64

Valores únicos por c

In [14]:
# 9. Estadísticas generales de las columnas numéricas
df.describe()

,Severity,Start_Time,End_Lat,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.436947e+06,6728423,4.134078e+06,7.436947e+06,7.436947e+06,7.436947e+06,7.436947e+06,7.436947e+06,7.436947e+06,7.436947e+06,7.436947e+06
mean,2.211633e+00,2020-03-18 10:32:32.996364544,3.623902e+01,5.537356e-01,6.168741e+01,5.829143e+01,6.485486e+01,2.954249e+01,9.092696e+00,7.691034e+00,8.018449e-03
min,1.000000e+00,2016-01-14 20:18:33,2.456601e+01,0.000000e+00,-4.500000e+01,-6.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,2018-09-13 06:43:02.500000,3.344132e+01,0.000000e+00,4.900000e+01,5.100000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,2020-06-17 23:51:25,3.614698e+01,2.700000e-02,6.400000e+01,5.829143e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,2021-10-26 15:10:19.500000,4.019984e+01,4.550000e-01,7.600000e+01,7.200000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.000000e+01,8.018449e-03
max,4.000000e+00,2023-03-31 23:30:00,4.907500e+01,4.417500e+02,1.960000e+02,1.960000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01
std,4.858861e-01,NaN,5.295181e+00,1.755476e+00,1.897791e+01,1.951898e+01,2.280500e+01,9.980924e-01,2.678072e+00,5.275012e+00,7.755930e-02


In [15]:
# 10. Guardar dataset limpio
os.makedirs("../data/processed", exist_ok=True)
df.to_csv(PROCESSED_PATH, index=False)
print(f"\nDataset limpio guardado en: {PROCESSED_PATH}")


Dataset limpio guardado en: ../data/processed/accidents_clean.csv


In [16]:
# Verificar la estructura y el tipo de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7436947 entries, 0 to 7728393
Data columns (total 32 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Severity           int64         
 1   Start_Time         datetime64[ns]
 2   End_Lat            float64       
 3   Distance(mi)       float64       
 4   City               object        
 5   County             object        
 6   State              object        
 7   Timezone           object        
 8   Temperature(F)     float64       
 9   Wind_Chill(F)      float64       
 10  Humidity(%)        float64       
 11  Pressure(in)       float64       
 12  Visibility(mi)     float64       
 13  Wind_Direction     object        
 14  Wind_Speed(mph)    float64       
 15  Precipitation(in)  float64       
 16  Weather_Condition  object        
 17  Amenity            bool          
 18  Bump               bool          
 19  Crossing           bool          
 20  Give_Way           bool      